In [43]:
import warnings
import findspark

In [44]:
findspark.init()

In [45]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [46]:
spark = SparkSession.builder.appName("Balance Calculator").getOrCreate()

In [105]:
df = spark.read.csv('./data/transaction_summary.csv', inferSchema=True, header=True)
df.printSchema()
df.show(5)

root
 |-- Account No: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- TRANSACTION DETAILS: string (nullable = true)
 |-- CHQ.NO.: integer (nullable = true)
 |-- VALUE DATE: string (nullable = true)
 |--  WITHDRAWAL AMT : string (nullable = true)
 |--  DEPOSIT AMT : string (nullable = true)

+-------------+---------+--------------------+-------+----------+----------------+--------------+
|   Account No|     DATE| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE| WITHDRAWAL AMT |  DEPOSIT AMT |
+-------------+---------+--------------------+-------+----------+----------------+--------------+
|409000611074'|29-Jun-17|TRF FROM  Indiafo...|   NULL| 29-Jun-17|            NULL| 1,000,000.00 |
|409000611074'|05-Jul-17|TRF FROM  Indiafo...|   NULL| 05-Jul-17|            NULL| 1,000,000.00 |
|409000611074'|18-Jul-17|FDRL/INTERNAL FUN...|   NULL| 18-Jul-17|            NULL|   500,000.00 |
|409000611074'|01-Aug-17|TRF FRM  Indiafor...|   NULL| 01-Aug-17|            NULL| 3,000,000.00 |
|409

In [106]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col

In [107]:
df = df.withColumn(' WITHDRAWAL AMT ', regexp_replace(' WITHDRAWAL AMT ', ',', ''))
df = df.withColumn(' DEPOSIT AMT ', regexp_replace(' DEPOSIT AMT ', ',', ''))
df = df.withColumn(' WITHDRAWAL AMT ', col(' WITHDRAWAL AMT ').cast(DoubleType()))
df = df.withColumn(' DEPOSIT AMT ', col(' DEPOSIT AMT ').cast(DoubleType()))
df.show(5)

+-------------+---------+--------------------+-------+----------+----------------+-------------+
|   Account No|     DATE| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |
+-------------+---------+--------------------+-------+----------+----------------+-------------+
|409000611074'|29-Jun-17|TRF FROM  Indiafo...|   NULL| 29-Jun-17|            NULL|    1000000.0|
|409000611074'|05-Jul-17|TRF FROM  Indiafo...|   NULL| 05-Jul-17|            NULL|    1000000.0|
|409000611074'|18-Jul-17|FDRL/INTERNAL FUN...|   NULL| 18-Jul-17|            NULL|     500000.0|
|409000611074'|01-Aug-17|TRF FRM  Indiafor...|   NULL| 01-Aug-17|            NULL|    3000000.0|
|409000611074'|16-Aug-17|FDRL/INTERNAL FUN...|   NULL| 16-Aug-17|            NULL|     500000.0|
+-------------+---------+--------------------+-------+----------+----------------+-------------+
only showing top 5 rows



In [108]:
df = df.withColumnRenamed(' WITHDRAWAL AMT ','wit')
df = df.withColumnRenamed(' DEPOSIT AMT ','dep')

In [109]:
df1 = df.fillna(0)

In [110]:
df1.show(5)

+-------------+---------+--------------------+-------+----------+---+---------+
|   Account No|     DATE| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE|wit|      dep|
+-------------+---------+--------------------+-------+----------+---+---------+
|409000611074'|29-Jun-17|TRF FROM  Indiafo...|      0| 29-Jun-17|0.0|1000000.0|
|409000611074'|05-Jul-17|TRF FROM  Indiafo...|      0| 05-Jul-17|0.0|1000000.0|
|409000611074'|18-Jul-17|FDRL/INTERNAL FUN...|      0| 18-Jul-17|0.0| 500000.0|
|409000611074'|01-Aug-17|TRF FRM  Indiafor...|      0| 01-Aug-17|0.0|3000000.0|
|409000611074'|16-Aug-17|FDRL/INTERNAL FUN...|      0| 16-Aug-17|0.0| 500000.0|
+-------------+---------+--------------------+-------+----------+---+---------+
only showing top 5 rows



In [111]:
df2 = df1.withColumn('Dep-Wit',df1.dep - df1.wit)
df2.show(5)

+-------------+---------+--------------------+-------+----------+---+---------+---------+
|   Account No|     DATE| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE|wit|      dep|  Dep-Wit|
+-------------+---------+--------------------+-------+----------+---+---------+---------+
|409000611074'|29-Jun-17|TRF FROM  Indiafo...|      0| 29-Jun-17|0.0|1000000.0|1000000.0|
|409000611074'|05-Jul-17|TRF FROM  Indiafo...|      0| 05-Jul-17|0.0|1000000.0|1000000.0|
|409000611074'|18-Jul-17|FDRL/INTERNAL FUN...|      0| 18-Jul-17|0.0| 500000.0| 500000.0|
|409000611074'|01-Aug-17|TRF FRM  Indiafor...|      0| 01-Aug-17|0.0|3000000.0|3000000.0|
|409000611074'|16-Aug-17|FDRL/INTERNAL FUN...|      0| 16-Aug-17|0.0| 500000.0| 500000.0|
+-------------+---------+--------------------+-------+----------+---+---------+---------+
only showing top 5 rows



In [112]:
df2 = df2.withColumn("Date", to_date(col("DATE"), "dd-MMM-yy"))

In [113]:
df2.show()

+-------------+----------+--------------------+-------+----------+--------+---------+---------+
|   Account No|      Date| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE|     wit|      dep|  Dep-Wit|
+-------------+----------+--------------------+-------+----------+--------+---------+---------+
|409000611074'|2017-06-29|TRF FROM  Indiafo...|      0| 29-Jun-17|     0.0|1000000.0|1000000.0|
|409000611074'|2017-07-05|TRF FROM  Indiafo...|      0| 05-Jul-17|     0.0|1000000.0|1000000.0|
|409000611074'|2017-07-18|FDRL/INTERNAL FUN...|      0| 18-Jul-17|     0.0| 500000.0| 500000.0|
|409000611074'|2017-08-01|TRF FRM  Indiafor...|      0| 01-Aug-17|     0.0|3000000.0|3000000.0|
|409000611074'|2017-08-16|FDRL/INTERNAL FUN...|      0| 16-Aug-17|     0.0| 500000.0| 500000.0|
|409000611074'|2017-08-16|FDRL/INTERNAL FUN...|      0| 16-Aug-17|     0.0| 500000.0| 500000.0|
|409000611074'|2017-08-16|FDRL/INTERNAL FUN...|      0| 16-Aug-17|     0.0| 500000.0| 500000.0|
|409000611074'|2017-08-16|FDRL/INTERNAL 

In [114]:
from pyspark.sql.window import Window

In [115]:
window = Window.partitionBy('Account No').orderBy("Date").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df2 = df2.withColumn("CumulativeSum", sum("Dep-Wit").over(window))

In [117]:
df2.show(500)

+----------+----------+--------------------+-------+----------+---------+---------+----------+-------------------+
|Account No|      Date| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE|      wit|      dep|   Dep-Wit|      CumulativeSum|
+----------+----------+--------------------+-------+----------+---------+---------+----------+-------------------+
|  1196428'|2015-01-01|DSB CASH PICKP In...|      0| 01-Jan-15|      0.0|1200000.0| 1200000.0|          1200000.0|
|  1196428'|2015-01-01|BEAT CASH PICKP D...|      0| 01-Jan-15|      0.0| 800000.0|  800000.0|          2000000.0|
|  1196428'|2015-01-02|CHQ DEP/45811/OWD...|      0| 02-Jan-15|      0.0|  15000.0|   15000.0|          2015000.0|
|  1196428'|2015-01-02|CHQ DEP/237843/OW...|      0| 02-Jan-15|      0.0|  25000.0|   25000.0|          2040000.0|
|  1196428'|2015-01-02|CHQ DEP/252242/OW...|      0| 02-Jan-15|      0.0|  25000.0|   25000.0|          2065000.0|
|  1196428'|2015-01-02|CHQ DEP/252243/OW...|      0| 02-Jan-15|      0.0|  25000

In [121]:
df2.write.csv('Balance_Calc.csv', header=True)

In [122]:
spark.stop()